In [1]:
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf

In [2]:
def load_images_from_folder(folders):
    class_images = []
    for folder in folders:
        images = []
        for filename in os.listdir(folder):
            img = Image.open(os.path.join(folder,filename))
            if img is not None:
                img = img.resize((299, 299))
                img_flip = ImageOps.flip(img)
                all_img = [img, img_flip]
                for i in all_img:
                    for r in [0, 90, 180, 270]:
                        images.append(np.array(i.rotate(r)))
        class_images.append(images)
    
    return class_images.copy()

In [3]:
def defined_input(class_images):
    features = []
    for images in class_images:
        for img in images:
            features.append(img)
    features = np.array(features)
    features = tf.keras.applications.inception_v3.preprocess_input(features, data_format=None)
    return features

In [4]:
#เตรียม output
def defined_output(class_images):
    label = []
    for count, images in enumerate(class_images):
        for i in range(len(images)):
            classes = len(class_images)
            output = np.zeros((classes,), dtype=int)
            output[count] = 1
            label.append(output)
    return np.array(label)

In [5]:
#change file path this.
foldersname = ['./images/class0 Cobra/Body',
               './images/class1 KingCobra/Body',
               './images/class2 BandedKrait/Body',
               './images/class3 MalayanKrait/Body',
               './images/class4 RussellViper/Body',
               './images/class5 MalayanPitViper/Body',
               './images/class6 WhiteLippedPitViper/Body',
               './images/class7 Big-eyed pit viper/Body',
               './images/class8 Trimeresurus [Popeia] phuketensis/Body',
               './images/class9 Shore pit viper/Body',
               './images/class10 Boiga nigriceps/Body',
               './images/class11 Gold-ringed cat snake/Body',
               './images/class12 Calliophis maculiceps/Body',
               './images/class13 Bungarus flaviceps/Body'
              ]
class_images = load_images_from_folder(foldersname)

X = np.copy(defined_input(class_images))
y = np.copy(defined_output(class_images))

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout , GlobalAveragePooling2D

In [8]:
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet', input_shape=(299,299,3))

base_model.trainable = False #Freeze Parameters

inputs = tf.keras.Input(shape=(299, 299, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(14, activation='softmax')(x)

In [9]:
model = Model(inputs, predictions)

In [10]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [11]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 299, 299, 3)]     0         
_________________________________________________________________
inception_v3 (Functional)    (None, 8, 8, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               524544    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)              

In [12]:
history = model.fit(X_train, y_train, batch_size = 32, epochs = 10, validation_split = 0.2)

Epoch 1/10
114/114 [==============================] - 735s 6s/step - loss: 2.2471 - accuracy: 0.2863 - val_loss: 1.5601 - val_accuracy: 0.5776
Epoch 2/10
114/114 [==============================] - 810s 7s/step - loss: 1.6389 - accuracy: 0.4827 - val_loss: 1.0665 - val_accuracy: 0.7030
Epoch 3/10
 59/114 [==============>...............] - ETA: 5:10 - loss: 1.3347 - accuracy: 0.5826

KeyboardInterrupt: 

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
base_model.trainable = True
fine_tune_at = 249

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate / 10),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size = 32, epochs = 20, initial_epoch = history.epoch[-1], validation_split = 0.2)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
model.save('./model/body_v3.h5')